In [1]:
# do not touch this cell 
%load_ext jupyter_record


# Imports


In [14]:
%%git_commit

import pandas as pd
from pandas.api.types import CategoricalDtype


# Dataset Load




In [22]:
%%git_commit

training = pd.read_csv('../archive/Combined_Flights_2018.csv').sample(1000000)
testing = pd.read_csv('../archive/Combined_Flights_2019.csv').sample(1000000)


In [23]:
%%git_commit

training = training.dropna()
testing = testing.dropna()





In [24]:
%%git_commit

dropped_column = ['Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline', 'DepTime', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'ActualElapsedTime', 'DepDelayMinutes', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'DistanceGroup', 'DivAirportLandings']




In [25]:
%%git_commit

for column in dropped_column:
    training = training.drop(column, axis = 1)
    testing = testing.drop(column, axis = 1)


In [17]:
%%git_commit


training['CRSDepTime'].unique()

array([1025, 1555, 1630, ...,  438,  317,  441])

# Sort by Appropriate Data

'Month', 'DayofMonth', 'CRSDepTime' -> not actually completely information free, but good enough

In [26]:
%%git_commit
training = training.sort_values(['Month', 'DayofMonth', 'CRSDepTime'])
testing = testing.sort_values(['Month', 'DayofMonth', 'CRSDepTime'])


In [27]:
%%git_commit

training


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,CRSElapsedTime,Distance,Year,...,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,CRSArrTime
356885,2018-01-01,United Air Lines Inc.,SFO,ORD,False,False,30,243.0,1846.0,2018,...,13930,1393006,30977,"Chicago, IL",IL,17,Illinois,41,0.0,633
110901,2018-01-01,United Air Lines Inc.,PHX,ORD,False,False,50,198.0,1440.0,2018,...,13930,1393006,30977,"Chicago, IL",IL,17,Illinois,41,0.0,508
222076,2018-01-01,Delta Air Lines Inc.,ANC,SEA,False,False,55,215.0,1448.0,2018,...,14747,1474703,30559,"Seattle, WA",WA,53,Washington,93,0.0,530
223183,2018-01-01,Delta Air Lines Inc.,FAI,SEA,False,False,100,240.0,1533.0,2018,...,14747,1474703,30559,"Seattle, WA",WA,53,Washington,93,0.0,600
329451,2018-01-01,JetBlue Airways,PSE,MCO,False,False,307,185.0,1179.0,2018,...,13204,1320402,31454,"Orlando, FL",FL,12,Florida,33,0.0,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2410121,2018-12-31,Spirit Air Lines,DEN,FLL,False,False,2358,223.0,1703.0,2018,...,11697,1169706,32467,"Fort Lauderdale, FL",FL,12,Florida,33,0.0,541
2123410,2018-12-31,JetBlue Airways,RNO,JFK,False,False,2359,305.0,2411.0,2018,...,12478,1247805,31703,"New York, NY",NY,36,New York,22,1.0,804
1783643,2018-12-31,United Air Lines Inc.,SFO,PHL,False,False,2359,323.0,2521.0,2018,...,14100,1410005,34100,"Philadelphia, PA",PA,42,Pennsylvania,23,0.0,822
2122225,2018-12-31,JetBlue Airways,SJU,JFK,False,False,2359,235.0,1598.0,2018,...,12478,1247805,31703,"New York, NY",NY,36,New York,22,1.0,254


# Time Series Data 


In [ ]:
%%git_commit




# Model Training



In [ ]:
%%git_commit


# Present to Students


In [ ]:
%%git_commit
